In [4]:
import graphviz
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.tools.eval_measures import rmse
from statsmodels.stats.outliers_influence import variance_inflation_factor # 다중공선성 패키지 불러오기
from sklearn.linear_model import LinearRegression # sklearn 선형회귀 모형 -> 변수 선택법을 위함
from sklearn.feature_selection import RFE # 변수 선택법 (후진 제거법)
from sklearn.preprocessing import scale # 데이터 scale
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
%matplotlib inline
matplotlib.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus']=False
data = pd.read_csv("./유방암.CSV",engine="python")
data.head()

,diagnosis,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# 데이터 결측치 확인

In [2]:
data.isnull().sum().head()

diagnosis         0
mean radius       0
mean texture      0
mean perimeter    0
mean area         0
dtype: int64

In [9]:
np.random.seed(seed=1234)
# 0.7 (70%)보다 작으면 True -> train 데이터, 아니면 False -> test 데이터
msk = np.random.rand(data.shape[0])<0.7
data_train = data[msk]
data_test = data[~msk]
# train/test 데이터의 목표변수 설명변수 지정
data_train_y=data_train["diagnosis"]
data_train_x=data_train.drop("diagnosis",axis = 1, inplace = False)
data_test_y = data_test["diagnosis"]
data_test_x = data_test.drop("diagnosis",axis =1, inplace=False)
# train 데이터와 test 데이터 크기
print("Train Data x size", data_train_x.shape)
print("Train Data y size", data_train_y.shape)
print("Test Data x size", data_test_x.shape)
print("Test Data y size", data_test_y.shape)

Train Data x size (385, 30)
Train Data y size (385,)
Test Data x size (184, 30)
Test Data y size (184,)


In [8]:
tree_uncustomized = DecisionTreeClassifier(random_state = 1234)
tree_uncustomized.fit(data_train_x, data_train_y)

print("Accuracy on training set : ",tree_uncustomized.score(data_train_x,data_train_y))

print("Accuracy on test set : ",tree_uncustomized.score(data_test_x,data_test_y))

Accuracy on training set :  1.0
Accuracy on test set :  0.9347826086956522
